In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')
from pandas import read_csv
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
plt.style.use('Solarize_Light2')
import seaborn as sns
from datetime import datetime as dat, timedelta
from pandas import datetime
from dateutil.relativedelta import relativedelta
import statsmodels as sm
import statsmodels.api as sm  
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import ADFTest
from pmdarima.arima import auto_arima
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.statespace.varmax import VARMAX
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [22]:
def timeseries_models(df,targetparameter,exogparameter ,algorithmn,ratio):
    train_size = int(len(df) * ratio)
    train, test = df[0:train_size], df[train_size:len(df)]
    #algorithm = input('Enter algorithm to use:')
    if algorithm == 'SimpleExponentialSmoothing':
        model= SimpleExpSmoothing(train[targetparameter])
        model_fit = model.fit(smoothing_level=.5)
        prediction=pd.DataFrame(model_fit.forecast(steps=len(test[targetparameter])),columns=[targetparameter])
        future_forecast=model_fit.forecast(steps=future_period)
        print(future_forecast)
        future_forecast=future_forecast.values

    elif algorithm == 'Holtlinear':
        model=Holt(train[targetparameter])
        model_fit=model.fit(smoothing_level=.5,smoothing_slope=.1)
        prediction=pd.DataFrame(model_fit.forecast(steps=len(test[targetparameter])),columns=[targetparameter])
        future_forecast=model_fit.forecast(steps=future_period)
        print(future_forecast)
        future_forecast = future_forecast.values

    elif algorithm =='Holtwinter':
        trend=input('Enter trend add or mul:')
        seasonal= input('Enter seasoanl add or mul:')
        m= int(input('value of seasonal_period:'))
        model = ExponentialSmoothing(train[targetparameter], trend=trend, seasonal=seasonal,seasonal_periods=m)
        model_fit = model.fit()
        prediction = pd.DataFrame(model_fit.forecast(steps=len(test[targetparameter])),columns=[targetparameter])
        future_forecast=model_fit.forecast(steps=future_period)
        future_forecast = future_forecast.values
        print(future_forecast)
    
    elif algorithm =='ARIMA':
        p = int(input('value of p:'))
        d = int(input('value of d:'))
        q = int(input('value of q:'))
        model = ARIMA(train[targetparameter], order=(p,d,q))
        model_fit = model.fit(disp=False)
        pred=model_fit.forecast(steps=len(test[targetparameter]))
        prediction=pd.DataFrame(pred[0],index=test.index,columns=targetparameter)
        future_forecast=model_fit.forecast(steps=future_period)
        future_forecast=future_forecast[0]
        print(future_forecast)
    
    elif algorithm =='SARIMA':
        p = int(input('value of p:'))
        d = int(input('value of d:'))
        q = int(input('value of q:'))
        P = int(input('value of p:'))
        D = int(input('value of d:'))
        Q = int(input('value of q:'))
        m= int(input('value of seasonal_period:'))
        model = SARIMAX(train[targetparameter], order=(p,d,q),seasonal_order=(P,D,Q,m),enforce_stationarity=True, enforce_invertibility=True)
        model_fit = model.fit(disp=False)
        pred=model_fit.forecast(steps=len(test[targetparameter]))
        prediction=pd.DataFrame(pred,index=test.index,columns=targetparameter)
        future_forecast=model_fit.forecast(steps=future_period)
        future_forecast=future_forecast.values
        print(future_forecast)
    
    elif algorithm =='ARIMAX':
        p = int(input('value of p:'))
        d = int(input('value of d:'))
        q = int(input('value of q:'))
        model = ARIMA(train[targetparameter],exog=train[exogparameter],order=(p,d,q))
        model_fit = model.fit(disp=False)
        pred=model_fit.forecast(steps=len(test[targetparameter]),exog=test[exogparameter])
        prediction=pd.DataFrame(pred[0],index=test.index,columns=targetparameter)
        future_forecast=model_fit.forecast(steps=future_period, exog=exog_forecast)
        future_forecast=future_forecast[0]
        print(future_forecast)
        
    elif algorithm =='SARIMAX':
        p = int(input('value of p:'))
        d = int(input('value of d:'))
        q = int(input('value of q:'))
        P = int(input('value of P:'))
        D = int(input('value of D:'))
        Q = int(input('value of Q:'))
        m= int(input('value of seasonal_period:'))
        model = SARIMAX(train[targetparameter], exog=train[exogparameter],order=(p, d, q),seasonal_order=(P,D,Q,m),enforce_stationarity=True, enforce_invertibility=True)
        model_fit = model.fit(disp=False)
        pred=model_fit.forecast(steps=len(test[targetparameter]),exog=test[exogparameter])
        prediction=pd.DataFrame(pred,index=test.index,columns=targetparameter)
        future_forecast=model_fit.forecast(steps=future_period, exog=exog_forecast)
        future_forecast=future_forecast.values
        print(future_forecast)

    elif algorithm =='AUTO_ARIMA':
        model = auto_arima(train[targetparameter],start_p=0, d=1, start_q=0, max_p=3, max_d=2, max_q=3,
                           seasonal=False, error_action='ignore', trace=True, supress_warning=True,
                           stepwise=True, random_state=20, n_fits=50)
        model.fit(train[targetparameter])
        prediction=pd.DataFrame(model.predict(n_periods=len(test[targetparameter])),index=test.index,columns=[targetparameter])
        future_forecast=model.predict(n_periods=future_period)
        print(future_forecast)
        
    elif algorithm == 'AUTO_SARIMA':
        m= int(input('value of seasonal_period:'))
        model = auto_arima(train[targetparameter],start_p=0, d=0, start_q=0, max_p=2, max_d=1, max_q=2,
                           start_P=0, D=0, start_Q=0, max_P=2, max_D=1, max_Q=2, m=m,
                           seasonal=True, error_action='ignore', trace=True, supress_warning=True,
                           stepwise=True, random_state=20, n_fits=50)
        model.fit(train[targetparameter])
        prediction=pd.DataFrame(model.predict(n_periods=len(test[targetparameter])),index=test.index,columns=targetparameter)
        future_forecast=model.predict(n_periods=future_period)
        
    elif algorithm =='VAR':
        model = VAR(endog=train[targetparameter])
        model_fit = model.fit()
        pred = model_fit.forecast(model_fit.y, steps=len(test[targetparameter]))
        prediction = pd.DataFrame(pred,index=test.index,columns=targetparameter)
        future_forecast=model_fit.forecast(model_fit.y, steps=future_period)
        
    elif algorithm =='VARMA':
        p=int(input('value of p:'))
        q=int(input('value of q:')) #trend = 'n','c','t','ct' # default is 'c'
        model = VARMAX(endog=train[targetparameter], order=(p,q),trend='c',enforce_stationarity=True, enforce_invertibility=True)
        model_fit = model.fit()
        prediction = model_fit.forecast(steps=len(test[targetparameter]),index=test.index)
        future_forecast = model_fit.forecast(steps=future_period)
        future_forecast=future_forecast.values
        
    elif algorithm =='VARMAX':
        p=int(input('value of p:'))
        q=int(input('value of q:'))  #trend = 'n','c','t','ct' # default is 'c'
        model=VARMAX(endog=train[targetparameter],exog=train[exogparameter],order=(p,q),trend='c',enforce_stationarity=True, enforce_invertibility=True)
        model_fit = model.fit()
        prediction=pd.DataFrame(model_fit.predict(steps=len(test[targetparameter]),exog=test[exogparameter],columns=targetparameter))
        future_forecast=model_fit.forecast(steps=future_period, exog=exog_forecast) 
    

    for i in targetparameter:
        print('MAE value for', i, 'is : ', mean_absolute_error(prediction, test[targetparameter]))
        print('RMSE value for', i, 'is : ', sqrt(mean_squared_error(prediction, test[targetparameter])))
        print('MAPE value for', i, 'is : ',np.mean(np.abs((test[targetparameter].values - prediction.values) / test[targetparameter].values)) * 100)
        
    return train,test,prediction,future_forecast

path='/home/rupeshr/Desktop/TSA_Python/dataset/time series dataset/archive/monthly-beer-production-in-austr.csv'
date='Month'
targetparameter=['Monthly beer production']
exogparameter=['Gain','Loss']
df=pd.read_csv(path)
df[date] = pd.to_datetime(df[date])
df = df.sort_values(by=date)
df.set_index(date, inplace=True)
df=df.astype(float)
algorithm ='Holtlinear' # exgo variable only supports by ARIMAX and SARIMAX models
ratio=0.80
future_period=60
train,test,prediction,future_forecast=timeseries_models(df,targetparameter,exogparameter,algorithm,ratio)
print(future_forecast)
print(train.tail())

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)


1987-09-01    138.696261
1987-10-01    137.328617
1987-11-01    135.960973
1987-12-01    134.593329
1988-01-01    133.225685
1988-02-01    131.858040
1988-03-01    130.490396
1988-04-01    129.122752
1988-05-01    127.755108
1988-06-01    126.387464
1988-07-01    125.019820
1988-08-01    123.652176
1988-09-01    122.284532
1988-10-01    120.916888
1988-11-01    119.549244
1988-12-01    118.181599
1989-01-01    116.813955
1989-02-01    115.446311
1989-03-01    114.078667
1989-04-01    112.711023
1989-05-01    111.343379
1989-06-01    109.975735
1989-07-01    108.608091
1989-08-01    107.240447
1989-09-01    105.872802
1989-10-01    104.505158
1989-11-01    103.137514
1989-12-01    101.769870
1990-01-01    100.402226
1990-02-01     99.034582
1990-03-01     97.666938
1990-04-01     96.299294
1990-05-01     94.931650
1990-06-01     93.564005
1990-07-01     92.196361
1990-08-01     90.828717
1990-09-01     89.461073
1990-10-01     88.093429
1990-11-01     86.725785
1990-12-01     85.358141


In [3]:
print(df.index[-1])

1995-08-01 00:00:00


In [18]:
def futureforecast(df,future_period,future_forecast,frequency):
    future_dates=pd.date_range(start=str(df.index[-1]), periods=future_period+1, freq=frequency,closed='right')
    if len(future_dates)== future_period:
        print('Success')
    else:
        future_dates=pd.date_range(start=str(df.index[-1]), periods=future_period, freq=frequency,closed='right')
    final_df = pd.DataFrame(future_forecast,columns=targetparameter)
    final_df[date] = future_dates
    final_df.set_index(date,inplace=True)
    return final_df

future_period=60 # frequency link provided below
frequency='MS' #https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-offset-aliases
final_df=futureforecast(df,future_period,future_forecast,frequency)
print(final_df)

Success
            Monthly beer production
Month                              
1995-09-01               149.668316
1995-10-01               154.118420
1995-11-01               156.354246
1995-12-01               157.528397
1996-01-01               158.193512
1996-02-01               158.614560
1996-03-01               158.918586
1996-04-01               159.166504
1996-05-01               159.387519
1996-06-01               159.595637
1996-07-01               159.797570
1996-08-01               159.996537
1996-09-01               160.194083
1996-10-01               160.390947
1996-11-01               160.587485
1996-12-01               160.783865
1997-01-01               160.980171
1997-02-01               161.176440
1997-03-01               161.372692
1997-04-01               161.568936
1997-05-01               161.765176
1997-06-01               161.961414
1997-07-01               162.157651
1997-08-01               162.353888
1997-09-01               162.550124
1997-10-01          

In [ ]:
plt.plot(train[targetparameter], label='Train')
plt.plot(test[targetparameter], label='Test')
plt.plot(prediction, label='Prediction')
plt.plot(final_df, label='Future')
plt.legend()
plt.tight_layout()
plt.show()

In [78]:
future_dates=pd.date_range(start='2021-03-31', periods=45, freq='YS',closed='right')
print(len(future_dates))

46


In [20]:
print(df.index[-1])
print(test.index[-1])
print(future_forecast)

2015-12-01 00:00:00
2015-12-01 00:00:00
                  Money      Spending
2011-10-01  9094.104866  10509.777727
2011-11-01  9037.724718  10614.712824
2011-12-01  8991.474966  10615.539269
2012-01-01  8833.212653  10466.827215
2012-02-01  8684.043727  10327.032063
2012-03-01  8542.481821  10194.362586
2012-04-01  8408.147602  10068.466808
2012-05-01  8280.671974   9948.998785
2012-06-01  8159.704763   9835.630342
2012-07-01  8044.913674   9728.050057
2012-08-01  7935.983378   9625.962409
2012-09-01  7832.614646   9529.086965
2012-10-01  7734.523525   9437.157610
2012-11-01  7641.440561   9349.921816
2012-12-01  7553.110056   9267.139948
2013-01-01  7469.289369   9188.584604
2013-02-01  7389.748245   9114.039995
2013-03-01  7314.268185   9043.301348
2013-04-01  7242.641849   8976.174346
2013-05-01  7174.672478   8912.474591
2013-06-01  7110.173364   8852.027102
2013-07-01  7048.967327   8794.665829
2013-08-01  6990.886236   8740.233204
2013-09-01  6935.770542   8688.579700
2013-10-01